In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2

# Parameters
input_shape = (224, 224, 3)
num_classes = 6
batch_size = 32
epochs_initial = 10
epochs_finetune = 10

# ✅ HSV preprocessing function with normalization
def preprocess_hsv(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype(np.float32)
    img[..., 0] = img[..., 0] / 179.0  # H: [0,179] → [0,1]
    img[..., 1] = img[..., 1] / 255.0  # S: [0,255] → [0,1]
    img[..., 2] = img[..., 2] / 255.0  # V: [0,255] → [0,1]
    return img

# Image generators with HSV preprocessing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_hsv)
val_datagen   = ImageDataGenerator(preprocessing_function=preprocess_hsv)
test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_hsv)

# Dataset paths
train_dir = '/kaggle/input/diffmango224x224/AMBADIFUSEDATASET224X224/train'
val_dir   = '/kaggle/input/diffmango224x224/AMBADIFUSEDATASET224X224/val'
test_dir  = '/kaggle/input/diffmango224x224/AMBADIFUSEDATASET224X224/test'

# Data loaders
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical', shuffle=False)

# Load EfficientNetB1 (no pretrained weights)
base_model = EfficientNetB1(weights=None, include_top=False, input_shape=input_shape)
base_model.trainable = False

# Classification head
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=output)

# Compile model (initial)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Phase 1: Train Frozen Base
print("\n🔧 Training with frozen EfficientNetB1 base...")
history1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs_initial
)

# Phase 2: Fine-Tune Model
print("\n🔧 Fine-tuning EfficientNetB1...")
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs_finetune
)

# Merge training history
history = {
    'accuracy': history1.history['accuracy'] + history2.history['accuracy'],
    'val_accuracy': history1.history['val_accuracy'] + history2.history['val_accuracy']
}

# 🎯 Evaluation
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\n✅ Test Accuracy after fine-tuning HSV: {test_accuracy:.4f}")

# 🔍 Classification Report
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
class_names = list(test_generator.class_indices.keys())

print("\n📄 Classification Report: EfficientNetB1 + HSV")
print(classification_report(y_true, y_pred, target_names=class_names))

# 📊 Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='YlGnBu', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix: EfficientNetB1 + HSV')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

# 📈 Accuracy Plot
plt.figure(figsize=(10, 5))
plt.plot(history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history['val_accuracy'], label='Validation Accuracy', marker='s')
plt.title('EfficientNetB1 + HSV: Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()


2025-07-23 18:11:20.485535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753294280.703910      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753294280.769351      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 46646 images belonging to 6 classes.
Found 9993 images belonging to 6 classes.
Found 10001 images belonging to 6 classes.


I0000 00:00:1753294316.365436      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ normalization[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_

...

Epoch 1/10


I0000 00:00:1753294346.598210     114 service.cc:148] XLA service 0x7ab3bc003ed0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753294346.599059     114 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753294349.782785     114 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/1458 ━━━━━━━━━━━━━━━━━━━━ 18:14:27 45s/step - accuracy: 0.1562 - loss: 1.7918

I0000 00:00:1753294365.317173     114 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1458/1458 ━━━━━━━━━━━━━━━━━━━━ 453s 280ms/step - accuracy: 0.1766 - loss: 1.7916 - val_accuracy: 0.1793 - val_loss: 1.7912
Epoch 2/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 200s 137ms/step - accuracy: 0.1779 - loss: 1.7912 - val_accuracy: 0.1793 - val_loss: 1.7911
Epoch 3/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 211s 145ms/step - accuracy: 0.1775 - loss: 1.7913 - val_accuracy: 0.1793 - val_loss: 1.7911
Epoch 4/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 216s 148ms/step - accuracy: 0.1810 - loss: 1.7909 - val_accuracy: 0.1793 - val_loss: 1.7911
Epoch 5/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 199s 137ms/step - accuracy: 0.1784 - loss: 1.7911 - val_accuracy: 0.1793 - val_loss: 1.7911
Epoch 6/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 205s 140ms/step - accuracy: 0.1809 - loss: 1.7910 - val_accuracy: 0.1793 - val_loss: 1.7911
Epoch 7/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 202s 139ms/step - accuracy: 0.1779 - loss: 1.7912 - val_accuracy: 0.1793 - val_loss: 1.7911
Epoch 8/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 214s 146ms/step - accuracy: 0.1